In [1]:
from sentence_transformers import SentenceTransformer, util
passage_encoder = SentenceTransformer('facebook-dpr-ctx_encoder-single-nq-base')

query_encoder = SentenceTransformer('facebook-dpr-question_encoder-single-nq-base')

In [2]:
import pandas as pd
df = pd.read_json('C:/Users/ngoph/Desktop/zalo_ai/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl', lines=True)

In [3]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [11]:
def normalize_text(text):
    from bs4 import BeautifulSoup
    import re
    def strip_html_tags(text):
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text(separator=" ")
        return stripped_text
    text = strip_html_tags(text)
    # remove 'BULLET::::-'
    text = re.sub(r'BULLET::::-', ' ', text)
    # remove = if more than 1
    text = re.sub(r'={2,}', ' ', text)
    # emove duplicate spaces
    text = re.sub(r'  +', ' ', text)
    # remove  newline characters
    text = re.sub(r'\s+', ' ', text)

    return text

In [12]:
df['text'] = df['text'].parallel_apply(normalize_text)

In [13]:
df.head(5)

,id,url,title,text,timestamp,revid
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,Trang Chính __NOEDITSECTION__,2022-05-12 12:46:53+00:00,68591979
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society Internet Society hay ISOC là ...,2022-01-20 07:59:10+00:00,67988747
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt Tiếng Việt, cũng gọi là tiếng Việt ...",2022-05-29 03:42:42+00:00,68660631
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio Ohio (viết tắt là OH, viết tắt cũ là O.) ...",2022-04-17 08:15:22+00:00,68482118
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California California (phát âm như ""Ca-li-phót...",2022-06-16 15:27:07+00:00,68738039


In [68]:
passages = df[['title','text']].parallel_apply(lambda x: ' [SEP] '.join(x), axis=1).tolist()

In [29]:
passage_embeddings = passage_encoder.encode(passages,show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [32]:
import numpy as np

In [61]:
query = "Châu Văn Minh sinh ngày mấy"
query_embedding = query_encoder.encode(query)

#Important: You must use dot-product, not cosine_similarity
scores = util.dot_score(query_embedding, passage_embeddings)[0].numpy()
scores = zip(range(len(scores)), scores)
scores = sorted(scores, key=lambda x: x[1], reverse=True)

for idx, score in scores[0:10]:
    print(temp_df.iloc[idx]['title'], "(Score: %.4f)" % (score))

Xuân Viên (định hướng) (Score: 80.0451)
Minh Chuẩn (Score: 79.4481)
Hồ Sĩ Tạo (định hướng) (Score: 78.0993)
Đại Đỗ (Score: 77.6928)
Tiết Tương Công (Score: 76.8388)
Le Dourn (Score: 76.7260)
Lâm Đồng (định hướng) (Score: 76.4727)
Lệch khúc xạ (Score: 75.7669)
Doanh thu (kế toán) (Score: 75.3769)
Bộ Huyệt (穴) (Score: 74.9923)
